In [ ]:
import pandas as pd
import numpy as np
import json
import os

In [ ]:
from utils import load_json, DotaTokenizer

project_root = os.path.dirname(os.getcwd())
hero_dpath = os.path.join(project_root, 'data', 'heroes.json')
match_dpath = os.path.join(project_root, 'data', 'main_metadata_2023.csv')
pickbans_dpath = os.path.join(project_root, 'data', 'picks_bans_2023.csv')

In [ ]:
picks_df = pd.read_csv(pickbans_dpath)
matches_df = pd.read_csv(match_dpath)
hero_metadata = load_json(hero_dpath)

results = np.array(matches_df)
picks = np.array(picks_df)

In [ ]:
special_tks = ['[PAD]', '[PICK]', '[BAN]', '[RADSTART]', '[DIRESTART]', '[RADWIN]', '[DIREWIN]']
tokenizer = DotaTokenizer(hero_metadata, special_tks)

In [ ]:
'''
picks_df columns: is_pick, hero_id, team, order, match_id
results_df = match_id, radiant_win
'''

In [ ]:
def make_sample_array(tokenizer: DotaTokenizer, picks: np.array, results: np.array, hero_metadata: dict) -> tuple[str, int]:
    
    pass

In [ ]:
def make_dataset(tokenizer: DotaTokenizer, picks: np.array, results: np.array, hero_metadata: dict) -> np.array:

    dataset = np.array([0, 0])

    # Transform into one single function
    umps, umrs = np.unique(picks[:, 4]), np.unique(results[:, 0]) # Col 4 represents 'match_id', Col 0 representes 'match_id'
    conflicting_matches = np.setxor1d(umps, umrs)
    mask_picks, mask_results = ~np.isin(picks[:, 4], conflicting_matches), ~np.isin(results[:, 0], conflicting_matches)
    picks, results = picks[mask_picks], results[mask_results]

    # Get rid of matches with less than 24 picks

    umps = np.unique(picks[:, 4]) # Recompute the indices since the original computation may include pruned indices
    matches_mask = 
    
    #samples = make_sample_array(tokenizer, picks, results, hero_metadata)

    return picks, results

In [8]:
picks, results = make_dataset(tokenizer, picks, results, hero_metadata)

In [ ]:
picks.shape